

egraph that person sent


grpah rewriting chr

Fair interleaving

Stack lookup
do a condiuctive lp
meson pttp


https://hsd-pbsa.de/veranstaltung/scryer-prolog-meetup-2025/

https://www.scryer.pl/

In [2]:
%%file /tmp/test.pl

hello(world).


Writing /tmp/test.pl


In [4]:
! scryer-prolog /tmp/test.pl -g 'hello(X), write(X), halt.'

world

In [17]:
! swipl -g "hello(X), write(X), halt." /tmp/test.pl

world

## Why Prolog?
https://news.ycombinator.com/item?id=31210466
https://github.com/mitsuchi/copl-in-prolog/
https://github.com/hsk/simpletapl/blob/master/prolog5/fullupdate.pl
Norvig
Bratko
Art of prolog https://cliplab.org/~logalg/doc/The_Art_of_Prolog.pdf
Power of prolog  https://www.metalevel.at/prolog
Clocksin and Mellish
lambda prolog examples (?)
minikanren stuff

Type checkers
evaluation semantics

Typeclass resolution.

Leantap
Leancop
nanocop
PTTP


term rewriting, e-unification
KB


I don't find sudokus compelling. I think you're better off with other techniques.

# CLP
freeze(X,goals)
-->
state has mapping Var -> suspgoals

CLP(set) using CLP(B) keys trie
or sorted lists of CLP(Z). disjoint union. separation.

CLP vs eager e-unify


In [ ]:
from typing import NamedTuple

class State(NamedTuple):
    vs  : list[smt.ExprRef]
    goals : list[smt.BoolRef]
    subst : dict[smt.ExprRef, smt.ExprRef]
    constrs : list[smt.BoolRef]

    def is_sat(self) -> bool:
        s = smt.Solver()
        s.add(*self.goals)
        s.add(*self.constrs)
        s.add(*[k == v for k, v in self.subst.items()])
        return s.check() == smt.sat
    def propagate(self):
        s = smt.Solver()
        s.add(*self.goals)
        s.add(*self.constrs)
        s.add(*[k == v for k, v in self.subst.items()])
        if s.check() == smt.sat:
            m = s.model()
            for d in m.consts():
                with s:
                    s.add(d == m.eval(d))
                    if s.check() == smt.unsat:
                        #do substitution 
                        lhs = d()
                        rhs = m[d]
                        if lhs in self.subst:
                            assert self.subst[lhs] == rhs
                        self.subst[lhs] = rhs
        else:
            return None
    def subst_add(self, x, t):
        new_vs = [v for v in self.vs if not v.eq(x)]
        assert len(new_vs) + 1 == len(self.vs)
        new_subst = {**{v: smt.substitute(e, (x,t)) for v,e in self.subst.items()}, x: t}
        new_goals = [smt.substitute(g, (x, t)) for g in self.goals]
        new_constrs = [smt.substitute(c, (x, t)) for c in self.constrs]
        return State(new_vs, new_goals, new_subst, new_constrs)


def horn(prog, goal):
    states : list[State] = [] # fifo, lifo, other
    while states:
        state = states.pop()
        if state.is_sat() is False:
            continue
        if not state.goals:
            yield state
            continue
        else:
            goal = state.goals.pop()
            if smt.is_and(goal):
                state.goals.extend(smt.children(goal))
                states.append(state)
                continue
            elif smt.is_or(goal):
                for disj in smt.children(goal):
                    new_state = State(state.goals + [disj], state.subst.copy(), state.constrs.copy())
                    states.append(new_state)
                continue
            elif goal.decl() in clp_preds:
                    # constraint logic predicate
                    state.constrs.append(goal)
                    states.append(state)
                    continue
                for prog in progs:
                    # unify
                    # do subst into everything



eunify

In [ ]:
decls = {
    
}

## CLP(Egraph)
Use CHR union find. Local state. Why didn't I use chr here?
https://www.philipzucker.com/harrop-checkpoint/

```
impl(eq(A,B),G) :- chr_add(A,B), G.  
unify_egraph(T1,T2) :- T1 = T2.
unify_egraph(T1,T2) :- find(T1) = find(T2). # ematch(T1), ematch(T1).

```


# Lambda
https://github.com/LPCIC/elpi

In [15]:
%%file /tmp/lam.elpi
pred a o:int.
a 1.

main :- a X, print X.

Overwriting /tmp/lam.elpi


In [16]:
! elpi -test /tmp/lam.elpi


Parsing time: 0.000

Compilation time: 0.000

Typechecking time: 0.004
Determinacy checking time: 0.004
1

Success:

Time: 0.000

Constraints:


State:



# Coinductive and PTTP
Stack inspection is key to coinductive logic programming, PTTP

# FLP